In [68]:
import os
from scipy.misc import imread
from datetime import datetime
import pickle
import scipy.io
import numpy as np
from numpy import linalg as LA
import time
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import accuracy_score
from okl_fct import okl, lrokl, mse, predict


def load_feature():
    """
    load features directly
    """   
    with open('sift_feature/hog_train.dat', 'rb') as f:
        hog_train = pickle.load(f)
    with open('sift_feature/hog_test.dat', 'rb') as f:
        hog_test = pickle.load(f)
    with open('sift_feature/sift_train.dat', 'rb') as f:
        sift_train = pickle.load(f)
    with open('sift_feature/sift_test.dat', 'rb') as f:
        sift_test = pickle.load(f)
    with open('sift_feature/y_train.dat', 'rb') as f:
        y_train = pickle.load(f)
    with open('sift_feature/y_test.dat', 'rb') as f:
        y_test = pickle.load(f)
    return hog_train, hog_test, sift_train, sift_test, y_test, y_train

hog_train, hog_test, sift_train, sift_test, y_test, y_train = load_feature()

In [69]:
print('size train: %i'%len(X_train))
print('size test: %i'%len(X_test))
print('dim hog: %i'%len(hog_train[0]))
print('dim sift: %i'%len(sift_train[0]))

size train: 7316
size test: 1829
dim hog: 1764
dim sift: 100


# HOG
### train

In [70]:
hog_train_arr=np.array(hog_train).reshape((len(hog_train),len(hog_train[0])))
hog_test_arr=np.array(hog_test).reshape((len(hog_test),len(hog_test[0])))
#sift_train_arr=np.array(sift_train).reshape((len(sift_train),len(sift_train[0]),len(sift_train[0][0])))
#sift_test_arr=np.array(sift_test).reshape((len(sift_test),len(sift_test[0]),len(sift_test[0][0])))
K_hog_train = rbf_kernel(hog_train_arr)
K_hog_test = rbf_kernel(hog_test_arr,hog_train_arr)

In [71]:
K_hog_test.shape

(1829, 7316)

In [72]:
lam=np.array([0.1])
p_list = [1]
model_hog = okl(K_hog_train,y_train,lam)

lambda = 0.1


### test(lambda = 0.1)

In [73]:
pred_test = predict(K_hog_test,model_hog[0]['C'],model_hog[0]['L'])
pred_test.shape

(1829, 102)

In [74]:
np.max(pred_test)

1.2192379832546953

In [75]:
pred_test_clf = np.zeros(pred_test.shape)
for k in range(len(pred_test)):
    n = np.argmax(pred_test[k])
    pred_test_clf[k,n]=1

In [76]:
accuracy_score(y_test,pred_test_clf)

0.2974302897758338

# SIFT

In [77]:
K_sift_train = rbf_kernel(sift_train)
K_sift_test = rbf_kernel(sift_test,sift_train)

In [79]:
y_train_sift = np.delete(y_train, (3545,6546), 0)

In [80]:
lam=np.array([20])
p_list = [1]
model_sift = okl(K_sift_train,y_train_sift,lam)

lambda = 20


### test(lambda = 20)

In [81]:
pred_test_sift = predict(K_sift_test,model_sift[0]['C'],model_sift[0]['L'])
pred_test_sift.shape

(1829, 102)

In [84]:
np.max(pred_test_sift)

0.55996683798839963

In [82]:
pred_test_clf_sift = np.zeros(pred_test_sift.shape)
for k in range(len(pred_test_sift)):
    n = np.argmax(pred_test_sift[k])
    pred_test_clf_sift[k,n]=1

In [83]:
accuracy_score(y_test,pred_test_clf_sift)

0.26899945325314378